In [174]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import platform
import datetime
import os
import math
import random

print(tf.__version__)

2.15.0


In [175]:
# Các bạn có thể tải dữ liệu bằng đường liên kết này
# !wget https://storage.googleapis.com/download.tensorflow.org/data/rps.zip -O /tmp/rockpaperscissors.zip

In [176]:
# Hoặc tải dữ liệu trong TensorFlow Dataset
DATASET_NAME = 'rock_paper_scissors'

(dataset_train_raw, dataset_test_raw), dataset_info = tfds.load(
    name=DATASET_NAME,
    data_dir='tmp',
    with_info=True,
    as_supervised=True,
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
)

In [177]:
NUM_TRAIN_EXAMPLES = dataset_info.splits['train'].num_examples
NUM_TEST_EXAMPLES = dataset_info.splits['test'].num_examples
NUM_CLASSES = dataset_info.features['label'].num_classes

print('Number of TRAIN examples:', NUM_TRAIN_EXAMPLES)
print('Number of TEST examples:', NUM_TEST_EXAMPLES)
print('Number of label classes:', NUM_CLASSES)

Number of TRAIN examples: 2520
Number of TEST examples: 372
Number of label classes: 3


In [178]:
Image_shape = list(dataset_train_raw.take(1))[0][0].shape
print(Image_shape)

(300, 300, 3)


In [179]:
new_image_shape = (224,224,3)

DATA PREPROCESSING

In [180]:
def ChuanHoaAnh(image, label):
  image = tf.cast(image, tf.float32)
  image = image / 255
  new_size = (150,150)
  image = tf.image.resize(image, new_size)
  return image, label

In [181]:
dataset_train_raw = dataset_train_raw.map(ChuanHoaAnh)
dataset_test_raw = dataset_test_raw.map(ChuanHoaAnh)

DATA AUGMENTATION

In [182]:
def augment_flip(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image

In [183]:
def augment_color(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.random_hue(image, max_delta=0.08)
    image = tf.image.random_saturation(image, lower=0.7, upper=1.3)
    image = tf.image.random_brightness(image, 0.05)
    image = tf.image.random_contrast(image, lower=0.8, upper=1)
    image = tf.clip_by_value(image, clip_value_min=0, clip_value_max=1)
    return image

In [184]:
def augment_rotation(image: tf.Tensor) -> tf.Tensor:
    # Rotate 0, 90, 180, 270 degrees
    return tf.image.rot90(
        image,
        tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    )

In [185]:
def augment_inversion(image: tf.Tensor) -> tf.Tensor:
    random = tf.random.uniform(shape=[], minval=0, maxval=1)
    if random > 0.5:
        image = tf.math.multiply(image, -1)
        image = tf.math.add(image, 1)
    return image

In [186]:
def augment_zoom(image: tf.Tensor, min_zoom=0.8, max_zoom=1.0) -> tf.Tensor:
    image_width, image_height, image_colors = image.shape
    crop_size = (image_width, image_height)

    # Generate crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(min_zoom, max_zoom, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize(
            [img],
            boxes=boxes,
            box_indices=np.zeros(len(scales)),
            crop_size=crop_size
        )
        # Return a random crop
        return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]

    choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: image, lambda: random_crop(image))

In [187]:
def augment_data(image, label):
    image = augment_flip(image)
    image = augment_color(image)
    image = augment_rotation(image)
    image = augment_zoom(image)
    image = augment_inversion(image)
    return image, label

In [188]:
dataset_info.splits['train'].num_examples

2520

In [189]:
data_augmented_train = dataset_train_raw.map(augment_data)

MODEL

In [190]:
NUM_TRAIN_EXAMPLES = dataset_info.splits['train'].num_examples
NUM_TEST_EXAMPLES = dataset_info.splits['test'].num_examples
NUM_CLASSES = dataset_info.features['label'].num_classes

print('Number of TRAIN examples:', NUM_TRAIN_EXAMPLES)
print('Number of TEST examples:', NUM_TEST_EXAMPLES)
print('Number of label classes:', NUM_CLASSES)

Number of TRAIN examples: 2520
Number of TEST examples: 372
Number of label classes: 3


In [191]:
BATCH_SIZE = 32
data_augmented_train = data_augmented_train.shuffle(buffer_size = NUM_TRAIN_EXAMPLES)
data_augmented_train = data_augmented_train.batch(BATCH_SIZE)
# Tối ưu trong quá trình training
dataset_train_augmented_shuffled = data_augmented_train.prefetch(
    buffer_size=tf.data.experimental.AUTOTUNE
)


dataset_test_raw = dataset_test_raw.batch(BATCH_SIZE)

MODEL

In [192]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [193]:
model = Sequential([
    Conv2D(64, (3,3), padding="same", input_shape=(150,150,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), padding="same", activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), padding="same", activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), padding="same", activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation="relu"),
    Dense(3, activation="softmax")
])

In [194]:
from tensorflow.keras.optimizers import RMSprop

rmsprop = RMSprop(learning_rate = 0.001)

In [195]:
early_stopping =tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [196]:
model.compile(optimizer = rmsprop, loss = "sparse_categorical_crossentropy",metrics = ['acc'])

In [197]:
step_per_epoch = dataset_info.splits['train'].num_examples // BATCH_SIZE
valid_step = dataset_info.splits['test'].num_examples // BATCH_SIZE

In [198]:
els = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [199]:
model.fit(
    x=dataset_train_augmented_shuffled.repeat(),
    validation_data=dataset_test_raw.repeat(),
    steps_per_epoch=step_per_epoch,
    validation_steps=step_per_epoch,
    epochs=10,
    use_multiprocessing=True,  # Sử dụng multiprocessing để tăng tốc độ huấn luyện và giảm áp lực lên bộ nhớ
    workers=2  # Số lượng worker sử dụng trong multiprocessing, có thể tăng hoặc giảm tùy thuộc vào tài nguyên có sẵn
)


Epoch 1/10
78/78 [==============================] - 25s 58ms/step - loss: 1.0514 - acc: 0.4539 - val_loss: 0.8700 - val_acc: 0.5908
Epoch 2/10
78/78 [==============================] - 19s 248ms/step - loss: 0.6449 - acc: 0.7327 - val_loss: 0.5944 - val_acc: 0.7125
Epoch 3/10
78/78 [==============================] - 19s 250ms/step - loss: 0.3328 - acc: 0.8654 - val_loss: 0.3628 - val_acc: 0.8284
Epoch 4/10
78/78 [==============================] - 19s 246ms/step - loss: 0.2237 - acc: 0.9176 - val_loss: 0.4138 - val_acc: 0.7475
Epoch 5/10
78/78 [==============================] - 19s 247ms/step - loss: 0.1739 - acc: 0.9473 - val_loss: 0.2546 - val_acc: 0.8667
Epoch 6/10
78/78 [==============================] - 19s 249ms/step - loss: 0.1052 - acc: 0.9610 - val_loss: 0.5034 - val_acc: 0.8713
Epoch 7/10
78/78 [==============================] - 19s 250ms/step - loss: 0.1140 - acc: 0.9638 - val_loss: 0.3389 - val_acc: 0.8424
Epoch 8/10
78/78 [==============================] - 17s 217ms/step - l